# 컴포넌트 상태에 따른 파이프라인 구성

일반적인 파이프라인을 구성하는 컴포넌트(노드)는 워크플로우에 모두 포함되어 실행됩니다. <br>
하지만 노드의 출력에 따라 서로 다른 노드에 입력으로 제공하고 싶은 경우가 있습니다. <br>
본 예제에서는 노드의 출력 상태에 따라 다른 워크플로우를 구성하는 파이프라인을 작성합니다.<br>
상태에 따라 다른 워크플르우가 수행되지만 수행되지 않은 노드들 또한 존재하기 때문에 다음과 같은 그래프로 구성됩니다.

<img src='condition_example_image/node_condition.png' widht = '750px'></img>

In [1]:
import kfp
from kfp import dsl
from kfp.components import func_to_container_op
from typing import NamedTuple

워크플로우 확인을 위하여 각 컴포넌트는 노드A~F로 구성하여 진행합니다. <br>
각 노드의 역할은 다음과 같습니다. <br>

> 컴포넌트 상태 파악을 위해 필요한 노드
>> 노드A : 1부터 10 사이의 수를 임의로 두개 출력합니다. 사칙연산 계산에 사용됩니다. <br>
>> 노드B : 임의로 사칙연산의 하나를 출력합니다. 두 수를 계산하기 위해 사용됩니다. <br>

> 사칙연산을 수행하는 노드 <br>
>> 노드C : 더하기를 수행합니다. <br>
>> 노드D : 빼기를 수행합니다. <br>
>> 노드E : 곱하기를 수행합니다. <br>
>> 노드F : 나누기를 수행합니다. <br>

위 그래프의 구성 노드들을 정의한 후 수행 가능한 워크플로우를 생각해 보면 다음과 같습니다. <br>
> case C : (A,B)-> C <br>
> case D : (A,B)-> D <br>
> case E : (A,B)-> E <br>
> case F : (A,B)-> F <br>

## 상태를 확인하는 컴포넌트

먼저 노드A,B를 작성합니다.<br>
노드A는 워크플로우 확인을 위한 출력과 임의로 생성하는 두개의 숫자 출력을 갖습니다. <br>워크플로우와 함께 두개의 숫자를 보내야 하기 때문에 출력 인수의 타입은 list를 사용합니다.

In [2]:
@func_to_container_op
def node_A() -> NamedTuple('Outputs', [('workflow', str), ('number', list)]):
    task_A = 'A'
    import random
    number1 = random.randint(1,10)
    number2 = random.randint(1,10)

    return (task_A, [number1, number2])

다음으로 노드B는 사칙연산 리스트에서 임의의 숫자를 통해 하나를 선택하여 출력하도록 작성합니다.

In [3]:
@func_to_container_op
def node_B() -> NamedTuple('Outputs', [('workflow', str), ('cal', str)]) :
    task_B = 'B'
    import random
    calculator_list = ['plus', 'minus', 'multiply', 'division']
    cal = calculator_list[random.randint(0,3)]
    return (task_B, cal)

## 사칙연산 컴포넌트

노드A,B에서 제공하는 사칙연산과 임의의 두개 숫자를 입력받아 계산을 수행하는 노드(컴포넌트)를 작성합니다.

In [4]:
@func_to_container_op
def node_C_plus(number : list, C1:str, C2:str) -> NamedTuple('Outputs', [('workflow', str), ('result', float)]) :
    task_C = f'{(C1, C2)} -> C_plus'
    print(task_C)
    result = sum(number)
    # result = int(number[0]) + int(number[1])
    return (task_C, result)

@func_to_container_op
def node_D_minus(number : list, D1:str, D2:str) -> NamedTuple('Outputs', [('workflow', str), ('result', float)]) :
    task_D = f'{(D1, D2)} -> D_minus'
    print(task_D)
    result = int(number[0]) - int(number[1])
    return (task_D, result)

@func_to_container_op
def node_E_multiply(number : list, E1:str, E2:str) -> NamedTuple('Outputs', [('workflow', str), ('result', float)]):
    task_E = f'{(E1, E2)} -> E_multiply'
    print(task_E)
    result = int(number[0]) * int(number[1])
    return (task_E, result)

@func_to_container_op
def node_F_division(number : list, F1:str, F2:str) -> NamedTuple('Outputs', [('workflow', str), ('result', float)]):
    task_F = f'{(F1, F2)} -> F_division'
    print(task_F)
    result = int(number[0]) / int(number[1])
    return (task_F, result)


## 파이프라인 작성

파이프라인을 작성하기 위한 노드(컴포넌트)들을 모두 작성하였습니다. <br>
이제 노드B의 출력 상태에 따라 서로 다른 사칙연산을 수행하는 파이프라인을 작성해야 합니다. <br>
컴포넌트의 인스턴스를 생성한 후 출력의 상태를 확인할 때는 <span style="font-weight:bold">dsl패키지에서 Condition</span>을 사용합니다.

In [5]:
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B()

    with dsl.Condition(node_B_task.outputs['cal'] == 'plus'):
        node_C_plus_task = node_C_plus(node_A_task.outputs['number'], node_A_task.outputs['workflow'], node_B_task.outputs['workflow'])
    with dsl.Condition(node_B_task.outputs['cal'] == 'minus'):
        node_D_minus_task = node_D_minus(node_A_task.outputs['number'], node_A_task.outputs['workflow'], node_B_task.outputs['workflow'])
    with dsl.Condition(node_B_task.outputs['cal'] == 'multiply'):
        node_E_multiply_task = node_E_multiply(node_A_task.outputs['number'], node_A_task.outputs['workflow'], node_B_task.outputs['workflow'])
    with dsl.Condition(node_B_task.outputs['cal'] == 'division'):
        node_F_division_task = node_F_division(node_A_task.outputs['number'], node_A_task.outputs['workflow'], node_B_task.outputs['workflow'])

## 파이프라인 결과화면

kubeflow pipeline에 컴포넌트의 상태를 확인을 추가하면 다음과 같이 상태 확인 노드가 추가되어 그려집니다.

<img src='condition_example_image/condition_kfp_example.png' width='850px'></img>

작성되어 있는 파이프라인을 실제로 실행하면 다음과 같은 결과가 생성됩니다. 실제로 수행되는 노드만 출력 결과를 가지는 것을 확인할 수 있습니다. <br>
임의로 생성된 3,9 와 division을 선택하여 수행되는 것을 확인할 수 있으며 나머지 노드들은 수행되지 않은 상태인 것을 확인할 수 있습니다.

<img src='condition_example_image/condition_result1.png' width='850px'></img>

수행되지 않은 노드들은 상태 확인에서 division과 일치하지 않았다는 알림 문구를 확인할 수 있습니다.

<img src='condition_example_image/condition_result2.png' width='850px'></img>